In [1]:
import pandas as pd
import json

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

with open('en_data2.json') as json_file:
    data = json.load(json_file)


# Get HS
bst = pd.read_excel('BIEU-THUE-XNK-2021.06.22.xlsx', header=4, converters={'B':str, 'C':str, 'D': str})
bst = bst[['B', 'C', 'D']]

bst.drop(bst[pd.isna(bst['B']) & pd.isna(bst['C'])].index, inplace = True)


def remove_useless_end(row):
    if str(row['B']).endswith('00'):
        return str(row['B'])[:-2]
    else:
        return row['B']

bst['B'] = bst.apply(lambda row: remove_useless_end(row), axis=1)

bst['temp'] = bst['B'].shift(-1)
bst['temp'] = bst['temp'].astype(str)


def remove_useless_temp(row):
    if str(row['temp']).endswith('0'):
        return str(row['temp'])[:-1]
    else:
        return row['temp']

bst['temp'] = bst.apply(lambda row: remove_useless_temp(row), axis=1)


def get_hs5(row):
    result = row['B']
    # if re.match('- ([a-zA-Z0-9])\w+', row['D']) and pd.isna(row['B']):
    #     result = row['temp'][0:5]
    if str(row['D']).startswith('- ') and pd.isna(row['B']):
        result = row['temp'][0:-1]
    else:
        result = row['B']
    return result

bst['B'] = bst.apply(lambda row: get_hs5(row), axis=1)


In [2]:
headings_lst = [heading for heading in bst['B'].unique() if len(str(heading)) == 4]

In [6]:
for section in data:
    for chapter in section['chapters']:
        for heading in chapter['headings']:
            for hs6 in heading['hs6']:
                hs6['id'] = hs6['id'].split('-')[0]
                if hs6['id'][4] != '.':
                    hs6['id'] = hs6['id'][:4] + '.' + hs6['id'][4:]
                try:
                    hs_to_lookup = ''.join(hs6['id'].split('.'))
                    hs_to_lookup = ''.join(hs_to_lookup.split(' '))
                    hs6['title'] = bst.loc[(bst['B']) == hs_to_lookup, ['C']].iloc[0]['C']
                except IndexError:
                    try:
                        hs6['title'] = bst.loc[(bst['B']) == hs_to_lookup + '0', ['C']].iloc[0]['C']
                    except IndexError:
                        if hs_to_lookup.endswith('-') or hs_to_lookup.endswith('0'):
                            try:
                                hs6['title'] = bst.loc[(bst['B']) == hs_to_lookup[:-1], ['C']].iloc[0]['C']
                            except IndexError:
                                print(hs_to_lookup)
                        else:
                            try:
                                hs6['title'] = bst.loc[(bst['B']) == hs_to_lookup.split('-')[0], ['C']].iloc[0]['C']
                            except IndexError:
                                print(hs_to_lookup)
                
                




220292


In [4]:
headings_lst

['0101',
 '0102',
 '0103',
 '0104',
 '0105',
 '0106',
 '0201',
 '0202',
 '0203',
 '0204',
 '0206',
 '0207',
 '0208',
 '0209',
 '0210',
 '0301',
 '0302',
 '0303',
 '0304',
 '0305',
 '0306',
 '0307',
 '0308',
 '0401',
 '0402',
 '0403',
 '0404',
 '0405',
 '0406',
 '0407',
 '0408',
 '0410',
 '0502',
 '0505',
 '0506',
 '0507',
 '0508',
 '0511',
 '0601',
 '0602',
 '0603',
 '0604',
 '0701',
 '0703',
 '0704',
 '0705',
 '0706',
 '0708',
 '0709',
 '0710',
 '0711',
 '0712',
 '0713',
 '0714',
 '0801',
 '0802',
 '0803',
 '0804',
 '0805',
 '0806',
 '0807',
 '0808',
 '0809',
 '0810',
 '0811',
 '0812',
 '0813',
 '0901',
 '0902',
 '0904',
 '0905',
 '0906',
 '0907',
 '0908',
 '0909',
 '0910',
 '1001',
 '1002',
 '1003',
 '1004',
 '1005',
 '1006',
 '1007',
 '1008',
 '1101',
 '1102',
 '1103',
 '1104',
 '1105',
 '1106',
 '1107',
 '1108',
 '1201',
 '1202',
 '1205',
 '1207',
 '1208',
 '1209',
 '1210',
 '1211',
 '1212',
 '1214',
 '1301',
 '1302',
 '1401',
 '1404',
 '1501',
 '1502',
 '1503',
 '1504',
 '1505',
 

In [5]:
with open('en.json', 'w') as f:
    json.dump(data, f)